# Sources and Measurements Example

This example shows how to upload measurements from an external repository into the Viking Analytics Vibration service and how to read this data.

In [1]:
import json
import os
import sys
import logging
from pathlib import Path

import pandas as pd
import requests
from requests import HTTPError

# import mvg library with python bindings to mvg-API
from mvg import MVG

Note that the `TOKEN` is used both for authorization and authentication. Thus, each unique token represents a unique user and each user has their own unique database on the VA vibration service.

**You need to insert your token received from Viking Analytics here:**

In [2]:
# Replace by your own Token
VALID_TOKEN = "Add token here"

This is Viking Analytics default logging setup which can be adapted to suit your needs. Log messages are printed from `mvg` library, see the source code for details.

In [3]:
root_logger = logging.getLogger()
root_logger.setLevel("INFO")
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(name)s - %(message)s")
stream_handler = logging.StreamHandler(stream=sys.stderr)
stream_handler.setFormatter(formatter)
root_logger.addHandler(stream_handler)

## Downloading the Data

We pick the data from one source from our public charlie repo https://github.com/vikinganalytics/va-data-charlie.git for convenience. Clone that repository to get access to the data.

In [4]:
!git clone https://github.com/vikinganalytics/va-data-charlie.git

fatal: destination path 'va-data-charlie' already exists and is not an empty directory.


We are going to use two of the sources from the charlie dataset with IDs `u0001` and `u0005`

In [5]:
REF_DB_PATH = Path.cwd() / "va-data-charlie" / "charlieDb" / "acc"
SOURCE_IDS = ["u0001", "u0005"]

## Connect to the API

Instantiate a session object with mvg library a session object basically caches the endpoint and the token, to simplify the the calls to the mvg library.

In [6]:
ENDPOINT = "https://api.beta.multiviz.com"
session = MVG(ENDPOINT, VALID_TOKEN)

We now check if the server is alive. The hello message contains, amongst others the API version.

In [7]:
hello_message = json.dumps(session.say_hello())
print(hello_message)

2021-03-18 10:54:16,959 - INFO - mvg.mvg - Getting API info for: https://api.beta.multiviz.com


{"api": {"name": "MultiViz Engine API", "version": "v0.0.13"}, "features": {"mode_id": true, "blacksheep": false, "emerging_mode": false, "on_off": false, "asset_type": false, "indicator_arrow": false}, "request_status": ["initialized", "accepted", "queued", "ongoing", "failed", "ready"], "traffic_limit": 10000000, "max_source_id_length": 255, "max_meta_size": 50000, "max_number_of_samples": 400000, "max_number_of_sources": 100, "max_number_of_measurements": 1000, "db_version": "1.0.0"}


## Check Database

We start by seeing if there are any sources in the database. A source represents a measurement source e.g. one specific channel of a measurement sensor. Note that sources cannot be used to represent a sensor with several channels. If this is required, the client side needs to take care of it.

Because we want to start at a clean slate we will list all sources (potentially none) and delete all of them

In [8]:
sources = session.list_sources()

print("Retrieved sources")
for src in sources:
    print(src)
    # While the list returned contains all information
    # about all known sources, it is also possible
    # to query for a single source by its id
    s_info = session.get_source(src['source_id'])
    print(f"Source info retrieved for one source: {s_info}")

2021-03-18 10:54:17,113 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:54:17,114 - INFO - mvg.mvg - listing sources


Retrieved sources


The example below revolves around a source with source id u0001

In [9]:
SOURCE_ID = SOURCE_IDS[0]

To make sure we start from a clean slate we delete our resource in case it exists. All information including measurements and analysis results will be removed

In [10]:
try:
    source = session.get_source(SOURCE_ID)
    print(f"Deleting {SOURCE_ID}")
    session.delete_source(SOURCE_ID)
except HTTPError:
    print(f"Source {SOURCE_ID} does not exist")

2021-03-18 10:54:17,256 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:54:17,257 - INFO - mvg.mvg - retrieving source with source id=u0001


Source u0001 does not exist


## Build measurements

Now we want to (re) build the source and the attached measurements from scratch

In [11]:
src_path = REF_DB_PATH / SOURCE_ID
m_file_name = REF_DB_PATH / SOURCE_ID / "meta.json"
with open(m_file_name, "r") as json_file:
    meta = json.load(json_file)
print("creating")
print(meta)

creating
{'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris'}


Create the source and check for it

In [12]:
session.create_source(SOURCE_ID, meta)  # create
source = session.get_source(SOURCE_ID)
print("Retrieved")
print(source)

2021-03-18 10:54:17,418 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:54:17,418 - INFO - mvg.mvg - creating source with source id=u0001
2021-03-18 10:54:17,419 - INFO - mvg.mvg - metadata: {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris'}
2021-03-18 10:54:17,552 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:54:17,553 - INFO - mvg.mvg - retrieving source with source id=u0001


Retrieved
{'source_id': 'u0001', 'meta': {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris'}}


Update the source

In [13]:
meta['updated'] = "YES! I have been updated"
session.update_source(SOURCE_ID, meta)  # update
source = session.get_source(SOURCE_ID)
print("Retrieved")
print(source)

2021-03-18 10:54:17,689 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:54:17,690 - INFO - mvg.mvg - updating source with source id=u0001
2021-03-18 10:54:17,691 - INFO - mvg.mvg - metadata: {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris', 'updated': 'YES! I have been updated'}
2021-03-18 10:54:17,831 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:54:17,831 - INFO - mvg.mvg - retrieving source with source id=u0001


Retrieved
{'source_id': 'u0001', 'meta': {'assetId': 'assetA', 'measPoint': 'mloc01', 'location': 'paris', 'updated': 'YES! I have been updated'}}


Upload measurements to source. Measurements are tied to sources, they consist of

- an array of floating point values
- timestamp when the values were sampled
- a field for the duration of the measurement
- meta information to be stored along the measurement

In [14]:
meas = {f.split(".")[0] for f in os.listdir(src_path)}
meas.remove("meta")

`meas` now contains a list of timestamps representing the measurements in our repo we upload from iterate over all of them

In [15]:
for m in meas:

    # samples file for one measurement
    TS_MEAS = str(m) + ".csv"  # filename
    TS_MEAS = REF_DB_PATH / SOURCE_ID / TS_MEAS  # path to file
    ts_df = pd.read_csv(TS_MEAS, names=['acc'])  # read csv into df
    accs = ts_df.iloc[:, 0].tolist()  # convert to list
    print(f"Read {len(accs)} samples")

    # meta information file for one measurement
    TS_META = str(m) + ".json"  # filename
    TS_META = REF_DB_PATH / SOURCE_ID / TS_META  # path
    with open(TS_META, "r") as json_file:  # read json
        meas_info = json.load(json_file)  # into dict
    print(f"Read meta: {meas_info}")

    # get duration and other meta info
    duration = meas_info['duration']
    meta_info = meas_info['meta']
    # add sampling rate, not required by vibration API
    # but may be used on client side
    # in general any information can be stored
    # along the actual samples
    meta_info['sampling_rate'] = len(accs)/duration
    # <<< end of code specific for repo

    # Upload measurements
    print(f"Uploading {TS_MEAS}")
    try:
        # see mvg for details on this call
        session.create_measurement(sid=SOURCE_ID,
                                   duration=duration,
                                   timestamp=m,
                                   data=accs,
                                   meta=meta_info)
    except HTTPError as exc:
        print("OUCH")
        print(exc)

2021-03-18 10:54:17,999 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:54:18,000 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-03-18 10:54:18,000 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-03-18 10:54:18,001 - INFO - mvg.mvg -   timestamp: 1572609660
2021-03-18 10:54:18,001 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading /home/oskar/viking_unix/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1572609660.csv


2021-03-18 10:54:19,166 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:54:19,167 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-03-18 10:54:19,167 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-03-18 10:54:19,168 - INFO - mvg.mvg -   timestamp: 1572955260
2021-03-18 10:54:19,168 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading /home/oskar/viking_unix/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1572955260.csv


2021-03-18 10:54:20,411 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:54:20,411 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-03-18 10:54:20,412 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-03-18 10:54:20,413 - INFO - mvg.mvg -   timestamp: 1574164860
2021-03-18 10:54:20,413 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


...

Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading /home/oskar/viking_unix/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1572523260.csv


2021-03-18 10:55:13,448 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:13,449 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-03-18 10:55:13,449 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-03-18 10:55:13,450 - INFO - mvg.mvg -   timestamp: 1572177660
2021-03-18 10:55:13,451 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading /home/oskar/viking_unix/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1572177660.csv


2021-03-18 10:55:14,425 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:14,426 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-03-18 10:55:14,427 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-03-18 10:55:14,427 - INFO - mvg.mvg -   timestamp: 1572087660
2021-03-18 10:55:14,428 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading /home/oskar/viking_unix/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0001/1572087660.csv


## Read the Measurements

Check if we actually created the measurements by reading them

In [16]:
m = session.read_measurements(SOURCE_ID)
print(f"Read {len(m)} stored measurements")

2021-03-18 10:55:15,659 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:15,660 - INFO - mvg.mvg - retrieving all measurements from source id=u0001
2021-03-18 10:55:15,824 - INFO - mvg.mvg - 50 measurements in database
2021-03-18 10:55:15,824 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:15,825 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:15,825 - INFO - mvg.mvg - timestamp=1570186860
2021-03-18 10:55:16,294 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:16,295 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:16,295 - INFO - mvg.mvg - timestamp=1570273260
2021-03-18 10:55:16,825 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:16,826 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:16,826 - INFO - mvg.mvg - timestamp=1570359660
2021-03-18 10:55:17,304 - INFO - mvg.mvg - endpoint h

2021-03-18 10:55:32,800 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:32,801 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:32,801 - INFO - mvg.mvg - timestamp=1573128060
2021-03-18 10:55:33,277 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:33,277 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:33,278 - INFO - mvg.mvg - timestamp=1573214460
2021-03-18 10:55:33,791 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:33,791 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:33,792 - INFO - mvg.mvg - timestamp=1573300860
2021-03-18 10:55:34,257 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:34,258 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:34,259 - INFO - mvg.mvg - timestamp=1573387260
2021-03-18 10:55:35,820 - INFO - mvg.mvg - endpoint https://api.

Read 50 stored measurements


It is also possible to read a specific measurement at a single timestamp let's get the timestamp of the first measurement

In [17]:
ts_0 = m[0]['timestamp']
meas_0 = session.read_single_measurement(SOURCE_ID, ts_0)
# we'll printout the returned measurement
print(f"source_id: {meas_0['source_id']}")
print(f"timestamp: {meas_0['timestamp']}")
print(f"duration:  {meas_0['duration']}")
print(f"meta:      {meas_0['meta']}")
print(f"data:      {meas_0['data'][1:3]}...")

2021-03-18 10:55:41,730 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:41,731 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:41,732 - INFO - mvg.mvg - timestamp=1570186860


source_id: u0001
timestamp: 1570186860
duration:  2.8672073400507907
meta:      {'sampling_rate': 13950.857142857141}
data:      [0.63897705078125, -0.55078125]...


We can update the meta information for a measurement at a single timestamp let's get the timestamp of the first measurement

In [18]:
ts_0 = m[0]['timestamp']
new_meta = meas_0['meta']
new_meta['updated'] = "YES!"
session.update_measurement(SOURCE_ID, ts_0, new_meta)
meas_0_u = session.read_single_measurement(SOURCE_ID, ts_0)

# we'll printout the returned measurement
print(f"source_id: {meas_0_u['source_id']}")
print(f"timestamp: {meas_0_u['timestamp']}")
print(f"duration:  {meas_0_u['duration']}")
print(f"meta:      {meas_0_u['meta']}")
print(f"data:      {meas_0_u['data'][1:3]}...")

2021-03-18 10:55:42,227 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:42,228 - INFO - mvg.mvg - deleting measurement for source id=u0001
2021-03-18 10:55:42,229 - INFO - mvg.mvg -   timestamp: 1570186860
2021-03-18 10:55:42,360 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:42,360 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:42,360 - INFO - mvg.mvg - timestamp=1570186860


source_id: u0001
timestamp: 1570186860
duration:  2.8672073400507907
meta:      {'sampling_rate': 13950.857142857141, 'updated': 'YES!'}
data:      [0.63897705078125, -0.55078125]...


Finally we delete the measurement

In [19]:
session.delete_measurement(SOURCE_ID, ts_0)

2021-03-18 10:55:42,872 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:42,874 - INFO - mvg.mvg - deleting measurement for source id=u0001
2021-03-18 10:55:42,874 - INFO - mvg.mvg -   timestamp: 1570186860


We check if it's actually deleted

In [20]:
try:
    meas_0 = session.read_single_measurement(SOURCE_ID, ts_0)
except HTTPError:
    print("Previously deleted measurement does not exist")

2021-03-18 10:55:43,025 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:43,025 - INFO - mvg.mvg - retrieving measurements from source id=u0001
2021-03-18 10:55:43,026 - INFO - mvg.mvg - timestamp=1570186860


Previously deleted measurement does not exist


If all went well we end up here now the source u0001 is in the database along with a number measurements

## Uploading another source

Here we show how to upload another source in a more compact way.

In [21]:
NEW_SOURCE = SOURCE_IDS[1]

try:
    print(f"Deleting {NEW_SOURCE}")
    session.delete_source(NEW_SOURCE)
except HTTPError:
    print(f"Source {NEW_SOURCE} does not exist")
    
src_path = REF_DB_PATH / NEW_SOURCE
m_file_name = REF_DB_PATH / NEW_SOURCE / "meta.json"
with open(m_file_name, "r") as json_file:
    meta = json.load(json_file)
session.create_source(NEW_SOURCE, meta)  # create

meas = {f.split(".")[0] for f in os.listdir(src_path)}
meas.remove("meta")

for m in meas:

    # samples file for one measurement
    TS_MEAS = str(m) + ".csv"  # filename
    TS_MEAS = REF_DB_PATH / NEW_SOURCE / TS_MEAS  # path to file
    ts_df = pd.read_csv(TS_MEAS, names=['acc'])  # read csv into df
    accs = ts_df.iloc[:, 0].tolist()  # convert to list
    print(f"Read {len(accs)} samples")

    # meta information file for one measurement
    TS_META = str(m) + ".json"  # filename
    TS_META = REF_DB_PATH / NEW_SOURCE / TS_META  # path
    with open(TS_META, "r") as json_file:  # read json
        meas_info = json.load(json_file)  # into dict
    print(f"Read meta: {meas_info}")

    duration = meas_info['duration']
    meta_info = meas_info['meta']
    meta_info['sampling_rate'] = len(accs)/duration

    print(f"Uploading {TS_MEAS}")
    try:
        # see mvg for details on this call
        session.create_measurement(sid=NEW_SOURCE,
                                   duration=duration,
                                   timestamp=m,
                                   data=accs,
                                   meta=meta_info)
    except HTTPError as exc:
        print(exc)



2021-03-18 10:55:43,170 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:43,170 - INFO - mvg.mvg - deleting source with source id=u0005
2021-03-18 10:55:43,315 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:43,315 - INFO - mvg.mvg - creating source with source id=u0005
2021-03-18 10:55:43,316 - INFO - mvg.mvg - metadata: {'assetId': 'assetE', 'measPoint': 'mloc01', 'location': 'london'}


Deleting u0005
Source u0005 does not exist


2021-03-18 10:55:43,459 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:43,459 - INFO - mvg.mvg - creating measurement from source id=u0005
2021-03-18 10:55:43,460 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-03-18 10:55:43,460 - INFO - mvg.mvg -   timestamp: 1572609660
2021-03-18 10:55:43,460 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading /home/oskar/viking_unix/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0005/1572609660.csv


2021-03-18 10:55:44,660 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:55:44,661 - INFO - mvg.mvg - creating measurement from source id=u0005
2021-03-18 10:55:44,661 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-03-18 10:55:44,662 - INFO - mvg.mvg -   timestamp: 1572955260
2021-03-18 10:55:44,663 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


...

Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading /home/oskar/viking_unix/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0005/1572523260.csv


2021-03-18 10:56:41,005 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:56:41,005 - INFO - mvg.mvg - creating measurement from source id=u0005
2021-03-18 10:56:41,006 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-03-18 10:56:41,006 - INFO - mvg.mvg -   timestamp: 1572177660
2021-03-18 10:56:41,007 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading /home/oskar/viking_unix/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0005/1572177660.csv


2021-03-18 10:56:42,316 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 10:56:42,317 - INFO - mvg.mvg - creating measurement from source id=u0005
2021-03-18 10:56:42,317 - INFO - mvg.mvg -   duration:  2.8672073400507907
2021-03-18 10:56:42,318 - INFO - mvg.mvg -   timestamp: 1572087660
2021-03-18 10:56:42,319 - INFO - mvg.mvg -   meta data: {'sampling_rate': 13950.857142857141}


Read 40000 samples
Read meta: {'duration': 2.8672073400507907, 'meta': {}}
Uploading /home/oskar/viking_unix/mvg/docs/source/content/examples/va-data-charlie/charlieDb/acc/u0005/1572087660.csv
